In [13]:
import pickle as pc
import numpy as np
import subprocess
import h5py
from multiprocessing import Pool

In [14]:
list_of_files = subprocess.getoutput("ls iccv09Data/labels/").split('\n')

In [48]:
len(list_of_files[0:740])

715

In [4]:
def open_file(path):
    """
        Takes a a text labels path and returns an text file object
    """
    txt = open(path,'r')
    y = txt.read()
    return y

In [18]:
def numpify(txt):
    """
        Takes a text object lable and returns an numpy array of it
    """
    img = []
    l = txt.split('\n')
    for i in l:
        k = i.split(' ')
        temp = []
        for j in k:
            temp.append(j)
        if temp != ['']:
            img.append(temp)
#     print(len(img))
#     print(len(img[0]))
    return np.array(img,dtype='uint8')

In [19]:
def one_hot_encodify(labels_mat):
    """
        Takes an 240*320 numpy array of labels and returns an one hot encoding array of dimension 8*240*320
    """
    cl = np.zeros((8,240,320))
    classes = [i for i in range(8)]
    for i in classes:
        h = np.reshape(labels_mat,(240*320))
        p = np.zeros((240*320))
        for j in range(len(h)):
            if h[j] == i:
                  p[j] = 1
        cl[i] = np.reshape(p,(240,320))
    return cl

In [43]:
len(y.split('\n'))

241

In [21]:
y=open_file("./iccv09Data/labels/0000047.regions.txt")

In [22]:
%time numpify(y)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 45.6 ms


array([[0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 5, 5],
       [0, 0, 0, ..., 5, 5, 5],
       ..., 
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2]], dtype=uint8)

In [23]:
%time one_hot_encodify(numpify(y)).shape

CPU times: user 5.84 s, sys: 0 ns, total: 5.84 s
Wall time: 5.83 s


(8, 240, 320)

In [6]:
one_hot_encodings = []

In [7]:
not_processed=[]

In [8]:
'./h5py/h5py_'+list_of_files[3]

'./h5py/h5py_0000072.regions.txt'

In [24]:
with h5py.File('./h5py/h5py_'+list_of_files[3],'w') as hf:
    hf.create_dataset('OHE', data=one_hot_encodify(numpify(open_file("./iccv09Data/labels/" + list_of_files[3]))),compression='gzip',compression_opts=9)

In [30]:
def oheify(list_of_files):
    for file in list_of_files:
#         print(one_hot_encodify(numpify(open_file("./iccv09Data/labels/" + file))))
        try:
            print("Processing file {}  ...".format(file))
            with h5py.File('./h5py/h5py_'+file,'w') as hf:
                hf.create_dataset('OHE', data=one_hot_encodify(numpify(open_file("./iccv09Data/labels/" + file))),compression='gzip',compression_opts=9)
        except ValueError:
            print(file)
            not_processed.append(file)
        except Exception as e:
            print('Exception : ',file, ' ' , e)

In [44]:
pool = Pool(processes=20)

In [50]:
for i in range(20):
    pool.apply_async(oheify,[list_of_files[40*i:40*(i+1)]])

In [51]:
len(not_processed)

0

In [81]:
processed_files = [i for i in list_of_files if 'h5py_'+i not in not_retain]

In [82]:
len(processed_files)

560

In [61]:
h55ied = subprocess.getoutput('ls h5py/').split('\n')

'36864'

In [73]:
not_retain = []
for i in h55ied:
    if int(subprocess.getoutput('stat --printf="%s" h5py/'+i)) < 1000:
        not_retain.append(i)

with open('processed_files.pkl','wb') as f:
    pc.dump(processed_files,f)

In [3]:
import pickle as pc
pro = []
with open('processed_files.pkl','rb') as f:
    pro = pc.load(f)

In [15]:
not_ther = [i for i in list_of_files if i not in pro]

In [25]:
for i in not_ther:
    subprocess.getoutput('rm iccv09Data/images/'+i.split('.')[0]+'.jpg')

In [24]:
p

"  File: 'iccv09Data/images/9005273.jpg'\n  Size: 11460     \tBlocks: 24         IO Block: 4096   regular file\nDevice: 805h/2053d\tInode: 3181262     Links: 1\nAccess: (0644/-rw-r--r--)  Uid: ( 1000/   dibya)   Gid: ( 1000/   dibya)\nAccess: 2018-04-30 13:54:16.903570721 +0530\nModify: 2009-07-08 05:20:32.000000000 +0530\nChange: 2018-01-15 22:40:01.435712532 +0530\n Birth: -"

In [10]:
pro[45].split('.')[0]+'.jpg'

'0102435.jpg'

In [86]:
for i in not_retain:
    subprocess.getoutput('rm h5py/'+i)

with open('1_hot_encodings_processed.pkl','wb') as f:
    pc.dump(one_hot_encodings,f)

In [46]:
u = np.array(one_hot_encodings)

np.save(open("test",'wb'),u)